In [208]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.api import qqplot,add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import warnings 
import graphviz
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
from scipy.stats import expon, reciprocal
from sklearn.inspection import permutation_importance

matplotlib.rc("font", family="NanumGothic")
matplotlib.rc("axes", unicode_minus = False)

In [209]:
df1 = pd.read_csv('vip모델링(불량고객제거+결혼+세부주소지).csv')
df2 = pd.read_csv('불량고객모델링.csv',index_col=0)
df3 = pd.read_csv('구독_전체데이터.csv',index_col=0)

In [210]:
def fun_plot_hist(data, var):
    plt.hist(data[data["VIP"] == 1][var], label = "VIP", alpha = 0.5)
    plt.hist(data[data["VIP"] == 0][var], label = "일반", alpha = 0.5)
    plt.title(var)
    plt.legend()

In [211]:
def fun_print_crosstab(data, var):
    return pd.crosstab(data["VIP"], data[var], margins=True)

In [212]:
# def fun_print_crosstab(data, var):
#     return pd.crosstab(data["불량회원"], data[var], margins=True)

In [213]:
df_cross=fun_print_crosstab(df1,'연령대')

In [214]:
df_cross_T = df_cross.copy()

In [215]:
df_crossT = df_cross_T.T

In [216]:
df_crossT

VIP,0,1,All
연령대,,,
20대,1358,39,1397
30대,4260,201,4461
40대,2946,174,3120
50대,1558,97,1655
60대 이상,1024,60,1084
All,11146,571,11717


In [217]:
df_crossT['비율']=df_crossT[1]/df_crossT['All']

In [218]:
df_crossT1 = df_crossT.iloc[:5,:2]

In [219]:
df_crossT1

VIP,0,1
연령대,,
20대,1358,39
30대,4260,201
40대,2946,174
50대,1558,97
60대 이상,1024,60


In [220]:
from scipy.stats import chi2_contingency

from scipy.stats import shapiro
from scipy import stats
import scipy
from statsmodels.stats.proportion import proportions_ztest

In [221]:
# 귀무가설 : 서로 독립이다

# 기각하므로 상관관계가 있음

chi, pval, dof, expected = stats.chi2_contingency(df_crossT1)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.5f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))


chi-square test
chisq:22.197
p:0.00018
degree pf freedom:{} 4
expected value:
{} [[1328.921   68.079]
 [4243.604  217.396]
 [2967.954  152.046]
 [1574.348   80.652]
 [1031.174   52.826]]


In [222]:
df_cross=fun_print_crosstab(df1,'성별')
df_cross

성별,남,여,All
VIP,,,
0,646,10500,11146
1,45,526,571
All,691,11026,11717


In [223]:
df_cross = df_cross.iloc[:2,:2]

In [224]:
# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 성별간에는 vip 상관관계가 있음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:3.888
p:0.049
degree pf freedom:{} 1
expected value:
{} [[  657.326 10488.674]
 [   33.674   537.326]]


In [225]:
df_cross=fun_print_crosstab(df1,'결혼')
df_cross1 = df_cross.iloc[:2,[0,2]]
df_cross1

결혼,기혼,미혼
VIP,,
0,4611,4116
1,228,223


In [226]:
# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 결혼 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross1)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:0.806
p:0.369
degree pf freedom:{} 1
expected value:
{} [[4601.215 4125.785]
 [ 237.785  213.215]]


In [227]:
df_cross=fun_print_crosstab(df1,'세부주소지')


# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))


chi-square test
chisq:219.905
p:1.000
degree pf freedom:{} 414
expected value:
{} [[4.09040e+01 1.72179e+02 4.75630e+01 7.61010e+01 2.94890e+01 1.31275e+02
  5.70800e+00 1.90300e+00 2.14986e+02 1.80740e+01 1.79790e+02 3.42460e+01
  2.85400e+00 3.51970e+01 1.90300e+00 4.28070e+01 1.25567e+02 3.80500e+00
  1.52200e+01 9.51000e-01 2.47330e+01 9.51000e-01 5.51740e+01 1.42690e+01
  1.14152e+02 1.80740e+01 1.71230e+01 4.18560e+01 4.75600e+00 1.71230e+01
  2.18790e+01 1.50300e+02 1.54105e+02 9.51000e-01 1.61720e+01 9.51300e+00
  8.46630e+01 1.23660e+01 7.51500e+01 5.70800e+00 2.37820e+01 5.42220e+01
  2.09279e+02 1.04640e+01 9.89320e+01 1.82643e+02 6.27840e+01 1.14150e+01
  4.75600e+00 7.13450e+01 1.33180e+01 1.04640e+01 1.36031e+02 6.27840e+01
  8.56100e+00 5.80270e+01 4.66120e+01 2.37820e+01 4.75630e+01 4.09040e+01
  1.14150e+01 8.56140e+01 8.08580e+01 5.23200e+01 2.85400e+00 1.66472e+02
  8.65650e+01 5.70800e+00 5.70800e+00 5.70800e+00 2.85400e+00 4.18560e+01
  8.56100e+00 2.18790e+01 2.85

In [232]:
df_cross=fun_print_crosstab(df1,'구독여부')
df_cross = df_cross.iloc[:2,:2]
df_cross

구독여부,False,True
VIP,,
0,7500,1570
1,386,81


In [233]:
# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:0.000
p:1.000
degree pf freedom:{} 1
expected value:
{} [[7499.845 1570.155]
 [ 386.155   80.845]]


# 불량

In [234]:
def fun_print_crosstab(data, var):
    return pd.crosstab(data["불량회원"], data[var], margins=True)

In [235]:
df_cross=fun_print_crosstab(df2,'연령대')

In [236]:
df_cross

연령대,20대,30대,40대,50대,60대 이상,All
불량회원,,,,,,
0,1367,4337,2989,1580,1044,11317
1,74,148,88,58,35,403
All,1441,4485,3077,1638,1079,11720


In [237]:
df_crossT1 = df_cross.iloc[:2,:5]
df_crossT1

연령대,20대,30대,40대,50대,60대 이상
불량회원,,,,,
0,1367,4337,2989,1580,1044
1,74,148,88,58,35


In [239]:


# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_crossT1)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))


chi-square test
chisq:16.032
p:0.003
degree pf freedom:{} 4
expected value:
{} [[1391.45  4330.78  2971.195 1581.676 1041.898]
 [  49.55   154.22   105.805   56.324   37.102]]


In [242]:
df_cross=fun_print_crosstab(df2,'성별')
df_cross = df_cross.iloc[:2,:2]
df_cross

성별,남,여
불량회원,,
0,655,10662
1,30,373


In [244]:
# df_cross=fun_print_crosstab(df2,'연령대')


# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.5f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))


chi-square test
chisq:1.651
p:0.19884
degree pf freedom:{} 1
expected value:
{} [[  661.446 10655.554]
 [   23.554   379.446]]


In [248]:
df_cross=fun_print_crosstab(df2,'결혼')
df_cross

In [249]:

# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))


chi-square test
chisq:2.999
p:0.083
degree pf freedom:{} 1
expected value:
{} [[ 2454.19 11301.81]
 [   90.81   418.19]]


In [251]:
df_cross=fun_print_crosstab(df2,'구독여부')
df_cross = df_cross.iloc[:2,:2]
df_cross

구독여부,False,True
불량회원,,
0,7628,1592
1,264,49


In [252]:
# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.5f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:0.445
p:0.50491
degree pf freedom:{} 1
expected value:
{} [[7632.879 1587.121]
 [ 259.121   53.879]]


# 구독 데이터

In [253]:
def fun_print_crosstab(data, var):
    return pd.crosstab(data["구독여부"], data[var], margins=True)

In [260]:
df_cross=fun_print_crosstab(df2,'연령대')
df_cross = df_cross.iloc[:2,:5]
df_cross

연령대,20대,30대,40대,50대,60대 이상
구독여부,,,,,
False,956,3030,2078,1116,712
True,202,581,434,257,167


In [261]:
# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:7.398
p:0.116
degree pf freedom:{} 4
expected value:
{} [[ 958.663 2989.406 2079.587 1136.653  727.69 ]
 [ 199.337  621.594  432.413  236.347  151.31 ]]


In [256]:
df_cross=fun_print_crosstab(df2,'성별')
df_cross = df_cross.iloc[:2,:2]
df_cross

성별,남,여
구독여부,,
False,422,7470
True,110,1531


In [257]:
# df_cross=fun_print_crosstab(df1,'결혼')
# df_cross

# 귀무가설 : 서로 독립이다
# 기각하므로 상관관계가 있음
# 주소지 간에는 vip 상관관계 없음

chi, pval, dof, expected = stats.chi2_contingency(df_cross)

print('chi-square test')
print('chisq:{0:0.3f}'.format(chi))
print('p:{0:0.3f}'.format(pval))
print('degree pf freedom:{}',format(dof))
print('expected value:\n{}',format(expected.round(3)))

chi-square test
chisq:4.487
p:0.034
degree pf freedom:{} 1
expected value:
{} [[ 440.422 7451.578]
 [  91.578 1549.422]]


In [ ]:
df_cross=fun_print_crosstab(df2,'성별')
df_cross = df_cross.iloc[:2,:2]
df_cross